# Download 
•	gene2go.gz from ftp://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz  
•	Entrez Gene info from ftp://ftp.ncbi.nih.gov/gene/DATA/gene_info.gz    
•	go from https://purl.obolibrary.org/obo/go/go-basic.obo

# Get GO_annotations-9606-inferred-allev.tsv from 
https://github.com/dhimmel/gene-ontology/blob/d57fd938f90c79152449f5cd23d3c438a19ac2f5/code/process.ipynb#L492 

In [1]:
import collections
import os

import pandas
import networkx

In [5]:
download_dir ='../data/GeneOntology'

In [3]:
remove_subsets = {
    'goantislim_grouping', # Grouping classes that can be excluded
    'gocheck_do_not_annotate' # Term not to be used for direct annotation
    'gocheck_do_not_manually_annotate', # Term not to be used for direct manual annotation
}

propagate_along = {'is_a', 'part_of'}

experimental_codes = {
    'EXP', # Inferred from Experiment
    'IDA', # Inferred from Direct Assay
    'IPI', # Inferred from Physical Interaction
    'IMP', # Inferred from Mutant Phenotype
    'IGI', # Inferred from Genetic Interaction
    'IEP', # Inferred from Expression Pattern
}

In [4]:
#pip install obonet

In [5]:
#pip install networkx #== 3.2.1 # networkx==2.2 require early version of python but there's no native build for python below 3.8 for Apple Silicon

## Read Gene Ontology graph

In [7]:
#https://github.com/dhimmel/gene-ontology/blob/d57fd938f90c79152449f5cd23d3c438a19ac2f5/code/process.ipynb
import networkx
import obonet # installation: https://github.com/dhimmel/obonet load OBO-formatted ontologies into networkx

path = os.path.join(download_dir, 'go-basic.obo') #'https://purl.obolibrary.org/obo/go/go-basic.obo' #data-version: releases/2024-04-24
with open(path) as read_file:
    graph = obonet.read_obo(read_file)
#networkx.info(graph)
print(graph.number_of_nodes())
print(graph.number_of_edges())

42255
81848


In [8]:
#go_df = utilities.graph_to_dataframe(graph)
rows = list()
for node, data in graph.nodes(data=True):
    rows.append((node, data['name'], data['namespace']))
go_df = pandas.DataFrame(rows, columns=['go_id', 'go_name', 'go_domain'])
go_df = go_df.sort_values('go_id')
    
go_df.head(2)

,go_id,go_name,go_domain
0,GO:0000001,mitochondrion inheritance,biological_process
1,GO:0000002,mitochondrial genome maintenance,biological_process


In [9]:
# Remove nodes that should not be annotated
remove_nodes = set()
for node, data in graph.nodes(data=True):
    if remove_subsets & set(data.get('subset', [])):
        remove_nodes.add(node)
        #graph.remove_node(node)

# Remove edges that should not be propagated along
remove_edges = []
for u, v, key in graph.edges(data=False, keys=True):
    if key not in propagate_along:
        remove_edges.append((u, v, key))

for u, v, key in remove_edges:
    graph.remove_edge(u, v, key)

assert networkx.is_directed_acyclic_graph(graph)
print(graph.number_of_nodes())
print(graph.number_of_edges())

42255
73512


In [10]:
graph.nodes['GO:0004449']

{'name': 'isocitrate dehydrogenase (NAD+) activity',
 'namespace': 'molecular_function',
 'def': '"Catalysis of the reaction: isocitrate + NAD+ = 2-oxoglutarate + CO2 + NADH." [RHEA:23632]',
 'synonym': ['"isocitrate dehydrogenase (NAD) activity" EXACT [EC:1.1.1.41]',
  '"isocitrate:NAD+ oxidoreductase (decarboxylating)" EXACT [EC:1.1.1.41]',
  '"NAD dependent isocitrate dehydrogenase activity" EXACT [EC:1.1.1.41]',
  '"NAD isocitrate dehydrogenase activity" EXACT [EC:1.1.1.41]',
  '"NAD isocitric dehydrogenase activity" EXACT [EC:1.1.1.41]',
  '"NAD-linked isocitrate dehydrogenase activity" EXACT [EC:1.1.1.41]',
  '"NAD-specific isocitrate dehydrogenase activity" EXACT [EC:1.1.1.41]',
  '"nicotinamide adenine dinucleotide isocitrate dehydrogenase activity" EXACT [EC:1.1.1.41]'],
 'xref': ['EC:1.1.1.41',
  'MetaCyc:ISOCITRATE-DEHYDROGENASE-NAD+-RXN',
  'Reactome:R-HSA-70967 "isocitrate + NAD+ => alpha-ketoglutarate + CO2 + NADH + H+ [IDH3]"',
  'RHEA:23632'],
 'is_a': ['GO:0004448']}

## Read Entrez Gene and annotations

In [11]:
# Read Entrez Gene info
import pandas 
import os
#download_dir ='../data'
path = os.path.join(download_dir, 'gene_info.gz') #'ftp://ftp.ncbi.nih.gov/gene/DATA/gene_info.gz'
column_names = [
    'tax_id',
    'GeneID',
    'Symbol',
    'LocusTag',
    'Synonyms',
    'dbXrefs',
    'chromosome',
    'map_location',
    'description',
    'type_of_gene',
    'Symbol_from_nomenclature_authority',
    'Full_name_from_nomenclature_authority',
    'Nomenclature_status',
    'Other_designations',
    'Modification_date',
]
dtype = {x: str for x in column_names}
for column in 'tax_id', 'GeneID':
    dtype[column] = int
gene_df = pandas.read_table(path, comment='#', names=column_names, na_values=['-'], dtype=dtype, index_col=False)
gene_df = gene_df[['GeneID', 'Symbol', 'type_of_gene', 'tax_id']]
gene_df = gene_df[gene_df['tax_id']==9606]
gene_df.head(2)

/var/folders/sg/983t9lp12bsfyjv9ylh6002m0000gt/T/ipykernel_99531/176760046.py:26: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_df = pandas.read_table(path, comment='#', names=column_names, na_values=['-'], dtype=dtype, index_col=False)
/var/folders/sg/983t9lp12bsfyjv9ylh6002m0000gt/T/ipykernel_99531/176760046.py:26: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  gene_df = pandas.read_table(path, comment='#', names=column_names, na_values=['-'], dtype=dtype, index_col=False)


,GeneID,Symbol,type_of_gene,tax_id
11465943,1,A1BG,protein-coding,9606
11465944,2,A2M,protein-coding,9606


In [12]:
#Download gene2go.gz from ftp://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz
#Unzip it
#Read annotations #https://github.com/dhimmel/gene-ontology/blob/d57fd938f90c79152449f5cd23d3c438a19ac2f5/code/utilities.py
import pandas 
import os

path = os.path.join(download_dir, 'gene2go.gz') #'ftp://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz'
column_names = ['tax_id','GeneID','GO_ID','Evidence','Qualifier','GO_term','PubMed','Category']
goa_df = pandas.read_table(path, comment='#', names=column_names, na_values=['-'], dtype=None, index_col=False)
goa_df = goa_df[goa_df['tax_id']==9606]
goa_df.head(2)

,tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
22059886,9606,1,GO:0002764,IBA,involved_in,immune response-regulating signaling pathway,NaN,Process
22059887,9606,1,GO:0003674,ND,enables,molecular_function,NaN,Function


## Add and propagate annotations

In [13]:
def is_NOT_qaulifier(qualifier):
    """
    Returns whether a `Qualifier` in gene2go is a NOT qualifier.
    http://geneontology.org/page/go-annotation-conventions#not
    """
    if pandas.isnull(qualifier):
        return False
    if not qualifier:
        return False
    if qualifier.upper().startswith('NOT'):
        return True
    return False

In [14]:
def annotate_graph(graph, goa_df):
    """Add direct annotations to graph"""
    graph = graph.copy()
    
    # Add dictionary items for storing annotations
    for node, data in graph.nodes.items():
        for key in 'direct_annotations', 'direct_not_annotations', 'inferred_annotations':
            data[key] = set()

    # Populate direct annotations
    for i, row in goa_df.iterrows():

        go_id = row['GO_ID']
        if go_id not in graph:
            continue

        key = 'direct_not_annotations' if is_NOT_qaulifier(row.Qualifier) else 'direct_annotations'

        gene = row['GeneID']
        graph.nodes[go_id][key].add(gene) #https://networkx.org/documentation/networkx-2.2/tutorial.html#node-attributes
    
    return graph

In [15]:
def propagate_annotations(graph):
    """Infer annotations through propagations"""
    for node in networkx.topological_sort(graph):
        data = graph.nodes[node]
        inferred = data['inferred_annotations']
        inferred -= data['direct_not_annotations']
        inferred |= data['direct_annotations']
        for subsuming_node in graph.successors(node):
            subsuming_data = graph.nodes[subsuming_node]
            subsuming_data['inferred_annotations'] |= inferred

In [16]:
joiner = lambda x: '|'.join(map(str, x))

def extract_annotation_df(graph):
    """Create an annotation dataframe"""
    rows = list()
    for node, data in graph.nodes.items():
        if node in remove_nodes:
            continue
        for kind in 'direct', 'inferred':
            for gene in data['{}_annotations'.format(kind)]:
                rows.append((node, kind, gene))
    
    annotation_df = pandas.DataFrame(rows, columns=['go_id', 'kind', 'GeneID'])
    annotation_df = annotation_df.merge(gene_df)

    rows = list()
    for (tax_id, kind), taxon_df in annotation_df.groupby(['tax_id', 'kind']):
        for go_id, term_df in taxon_df.groupby('go_id'):
            term_df = term_df.sort_values('GeneID')
            row = tax_id, go_id, kind, len(term_df), joiner(term_df['GeneID']), joiner(term_df['Symbol'])
            rows.append(row)
    wide_df = pandas.DataFrame(rows, columns = ['tax_id', 'go_id', 'annotation_type', 'size', 'gene_ids', 'gene_symbols'])
    wide_df = go_df.merge(wide_df)
    return wide_df

## Extract and save annotations

In [17]:
"""
graph_annot = annotate_graph(graph, goa_df)

propagate_annotations(graph_annot)

graph_annot.nodes['GO:0004449']['direct_annotations']

rows = list()
for node, data in graph_annot.nodes.items():
    if node in remove_nodes:
        continue
    for gene in data['inferred_annotations']:
        kind ='inferred_annotations'
        rows.append((node, kind, gene))

annotation_df = pandas.DataFrame(rows, columns=['go_id', 'kind', 'GeneID'])
annotation_df = annotation_df.merge(gene_df)
annotation_df
"""

"\ngraph_annot = annotate_graph(graph, goa_df)\n\npropagate_annotations(graph_annot)\n\ngraph_annot.nodes['GO:0004449']['direct_annotations']\n\nrows = list()\nfor node, data in graph_annot.nodes.items():\n    if node in remove_nodes:\n        continue\n    for gene in data['inferred_annotations']:\n        kind ='inferred_annotations'\n        rows.append((node, kind, gene))\n\nannotation_df = pandas.DataFrame(rows, columns=['go_id', 'kind', 'GeneID'])\nannotation_df = annotation_df.merge(gene_df)\nannotation_df\n"

In [18]:
#Get GO_annotations-9606-inferred-allev.tsv from https://github.com/dhimmel/gene-ontology/blob/d57fd938f90c79152449f5cd23d3c438a19ac2f5/code/process.ipynb#L492 
env_type='allev'
graph_annot = annotate_graph(graph, goa_df)
propagate_annotations(graph_annot)
annotation_df = extract_annotation_df(graph_annot)
annotation_df

,go_id,go_name,go_domain,tax_id,annotation_type,size,gene_ids,gene_symbols
0,GO:0000002,mitochondrial genome maintenance,biological_process,9606,direct,11,291|1890|4205|4358|4976|9361|10000|55186|80119...,SLC25A4|TYMP|MEF2A|MPV17|OPA1|LONP1|AKT3|SLC25...
1,GO:0000002,mitochondrial genome maintenance,biological_process,9606,inferred,26,142|291|1763|1890|2021|4205|4358|4976|5428|624...,PARP1|SLC25A4|DNA2|TYMP|ENDOG|MEF2A|MPV17|OPA1...
2,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,9606,direct,2,55650|79087,PIGV|ALG12
3,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,9606,inferred,2,55650|79087,PIGV|ALG12
4,GO:0000012,single strand break repair,biological_process,9606,direct,10,1161|2074|3981|7141|7515|23411|54840|55775|200...,ERCC8|ERCC6|LIG4|TNP1|XRCC1|SIRT1|APTX|TDP1|AP...
...,...,...,...,...,...,...,...,...
40975,GO:2001304,lipoxin B4 metabolic process,biological_process,9606,inferred,1,239,ALOX12
40976,GO:2001306,lipoxin B4 biosynthetic process,biological_process,9606,direct,1,239,ALOX12
40977,GO:2001306,lipoxin B4 biosynthetic process,biological_process,9606,inferred,1,239,ALOX12
40978,GO:2001311,lysobisphosphatidic acid metabolic process,biological_process,9606,direct,2,51205|57406,ACP6|ABHD6


In [19]:
df = annotation_df[(annotation_df['tax_id']==9606)&(annotation_df['annotation_type']=='inferred')]
path = os.path.join(download_dir, 'GO_annotations-9606-inferred-allev.tsv') 
df.to_csv(path, sep='\t', index=False)

In [ ]:
"""
for ev_type in 'allev', 'expev':
    goa_subset_df = goa_df
    if ev_type == 'expev':
        goa_subset_df = goa_subset_df[goa_subset_df.Evidence.isin(experimental_codes)]
    graph_annot = annotate_graph(graph, goa_subset_df)
    propagate_annotations(graph_annot)
    annotation_df = extract_annotation_df(graph_annot)

    for (tax_id, annotation_type), df in annotation_df.groupby(['tax_id', 'annotation_type']):
        path = utilities.get_annotation_path(annotation_dir, tax_id, annotation_type, ev_type, mkdir=True)
        print(path)
        df.to_csv(path, sep='\t', index=False)
"""

## Compare with previous version

In [22]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
# stats
import pandas as pd
df = pd.read_table('../data/GeneOntology/GO_annotations-9606-inferred-allev.tsv')
df.shape

(22283, 8)

In [26]:
import gzip
path = '../data/EntrezGene/Homo_sapiens.gene_info.gz'
read_file = gzip.open(path, 'rb')
gene_df = pd.read_table(read_file, na_values=['-'])
gene_df = gene_df.rename(columns={'#tax_id':'tax_id'})
gene_df = gene_df.query('tax_id == 9606')
columns = ['tax_id', 'GeneID', 'Symbol', 'chromosome', 'map_location', 'type_of_gene', 'description']
gene_df = gene_df[columns]
#gene_df.to_csv('data/genes-human.tsv', sep='\t', index=False)
gene_df = gene_df[gene_df.type_of_gene == 'protein-coding']
coding_genes = set(gene_df.GeneID)
len(coding_genes)

/var/folders/sg/983t9lp12bsfyjv9ylh6002m0000gt/T/ipykernel_99531/2484383344.py:4: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_df = pd.read_table(read_file, na_values=['-'])


20608

In [27]:
nodes =[]
edges =[]
for i, row in df.iterrows():
    genes = coding_genes & set(map(int, row.gene_ids.split('|')))
    if 2 > len(genes) or len(genes) > 1000:
        continue
    kind = row['go_domain'].replace('_', ' ').title()
    identifier = row['go_id']
    name=row['go_name']
    nodes.append([identifier, name, kind])
    for gene in genes:
        source_id = gene
        target_id = row['go_id']
        edge = 'Gene–participates–' + kind
        edges.append([source_id, target_id, edge])
        
nodes_df = pd.DataFrame(nodes, columns=['go_id', 'go_name', 'go_domain'])    
nodes_df.groupby('go_domain')['go_id'].nunique()

edges_df = pd.DataFrame(edges, columns=['gene', 'go_id', 'edge'])    
edges_df.groupby('edge').size()

go_domain
Biological Process    12322
Cellular Component     1695
Molecular Function     3460
Name: go_id, dtype: int64

edge
Gene–participates–Biological Process    548342
Gene–participates–Cellular Component     88885
Gene–participates–Molecular Function    104777
dtype: int64

In [28]:
#compare with hetionet version  438 mesh terms 
df_hetionet = pd.read_table('https://raw.githubusercontent.com/dhimmel/gene-ontology/87bab297f55db283e65a7a984607316b409415ae/annotations/taxid_9606/GO_annotations-9606-inferred-allev.tsv')

In [29]:
gene_df = pd.read_table('https://raw.githubusercontent.com/dhimmel/entrez-gene/a7362748a34211e5df6f2d185bb3246279760546/data/genes-human.tsv')
gene_df = gene_df[gene_df.type_of_gene == 'protein-coding']
coding_genes = set(gene_df.GeneID)
len(coding_genes)

20945

In [30]:
nodes =[]
edges =[]
for i, row in df_hetionet.iterrows():
    genes = coding_genes & set(map(int, row.gene_ids.split('|')))
    if 2 > len(genes) or len(genes) > 1000:
        continue
    kind = row['go_domain'].replace('_', ' ').title()
    identifier = row['go_id']
    name=row['go_name']
    nodes.append([identifier, name, kind])
    for gene in genes:
        source_id = gene
        target_id = row['go_id']
        edge = 'Gene–participates–' + kind
        edges.append([source_id, target_id, edge])
        
nodes_df = pd.DataFrame(nodes, columns=['go_id', 'go_name', 'go_domain'])    
nodes_df.groupby('go_domain')['go_id'].nunique()

edges_df = pd.DataFrame(edges, columns=['gene', 'go_id', 'edge'])    
edges_df.groupby('edge').size()

go_domain
Biological Process    11381
Cellular Component     1391
Molecular Function     2884
Name: go_id, dtype: int64

edge
Gene–participates–Biological Process    559504
Gene–participates–Cellular Component     73566
Gene–participates–Molecular Function     97222
dtype: int64